In [1]:
# Required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt


In [66]:
ventas_proveedor1.head()

,Sku,Date,Store_Id,Units,Price,Cadena
0,4220015,2017-01-01 SO Diaria Piezas,18,6.0,15.6,proveedor_1
1,4220015,2017-01-01 SO Diaria Piezas,24,0.0,0.0,proveedor_1
2,4220015,2017-01-01 SO Diaria Piezas,39,0.0,0.0,proveedor_1
3,4220015,2017-01-01 SO Diaria Piezas,44,0.0,0.0,proveedor_1
4,4220015,2017-01-01 SO Diaria Piezas,46,0.0,0.0,proveedor_1


In [69]:
ventas_proveedor2.head()

,Sku,Date,Store_Id,Pieces,Price,Cadena
0,4240039,2016/01/01,1000,23.0,$16.00,proveedor_2
1,4240039,2016/01/01,1001,116.0,$16.00,proveedor_2
2,4240039,2016/01/01,1002,146.0,$16.00,proveedor_2
3,4240039,2016/01/01,1003,17.0,$16.80,proveedor_2
4,4240039,2016/01/01,1004,5.0,$16.70,proveedor_2


In [65]:
# Carga de datos

clientes = pd.read_csv('maestro_clientes_prueba.csv', sep = ';',low_memory=False)
productos = pd.read_csv('maestro_productos_prueba.csv', sep = ',',low_memory=False)
ventas_proveedor1 = pd.read_csv('sellout_proveedor1.csv', sep = ',',low_memory=False)
ventas_proveedor2 = pd.read_csv('sellout_proveedor2.csv', sep = ',',low_memory=False)

##
clientes.columns = ['Store_Id','Nombre_tienda','Cadena','Formato','Municipio','Estado','Division','Distribuidor','Estructura','Grado']
ventas_proveedor1['Cadena'] = 'proveedor_1'
ventas_proveedor2['Cadena'] = 'proveedor_2'
ventas_df = pd.concat([ventas_proveedor1,ventas_proveedor2])

print("dim clientes: " + str(clientes.shape))
print(clientes.head(2) , '\n')

print("dim productos: " + str(productos.shape))
print(productos.head(2) , '\n')

print("dim ventas: " + str(ventas_df.shape))
print(ventas_df.head(2) , '\n')

dim clientes: (2452, 10)
   Store_Id              Nombre_tienda       Cadena Formato  \
0         2  Bloody Sea Green BookShop  proveedor_1    Shop   
1         4  Funny Sky Blue GipsyTrade  proveedor_1    Shop   

            Municipio   Estado Division    Distribuidor      Estructura Grado  
0  Bizarre White Shop  Coalmar  SURESTE  Mogulval Corp.  DISTRIBUIDORES    VI  
1  Bizarre White Shop  Coalmar  SURESTE  Mogulval Corp.  DISTRIBUIDORES    VI   

dim productos: (1242, 4)
     Sku      Sku_Description   Sku_Category Sku_Subcategory
0  11058       Frosted Mango      Sugar-free     Cereal bars
1  29207  Nut Cheerios Mango   Super-flavour      Cereal box 

dim ventas: (6068394, 7)
       Sku                         Date  Store_Id  Units Price       Cadena  \
0  4220015  2017-01-01 SO Diaria Piezas        18    6.0  15.6  proveedor_1   
1  4220015  2017-01-01 SO Diaria Piezas        24    0.0   0.0  proveedor_1   

   Pieces  
0     NaN  
1     NaN   



In [64]:
ventas[(ventas['Cadena']=='proveedor_1') & (ventas['Units'].isna())]

,Sku,Date,Store_Id,Units,Price,Cadena,Pieces,Sku_Description,Sku_Category,Sku_Subcategory,Nombre_tienda,Formato,Municipio,Estado,Division,Distribuidor,Estructura,Grado


In [52]:
ventas.shape

(6073832, 18)

In [62]:
# Unir dataset e productos con cada proveedor
clientes['Cadena'] = clientes['Cadena'].str.strip()

ventas = ventas_df.merge(productos, on="Sku", how="left").merge(clientes, on=["Store_Id","Cadena"], how="left")

# Check for missing values
print(ventas.isnull().sum())

Sku                      0
Date                     0
Store_Id                 0
Units              4862352
Price                    0
Cadena                   0
Pieces             1211480
Sku_Description        465
Sku_Category           465
Sku_Subcategory        465
Nombre_tienda        34100
Formato              34100
Municipio            34100
Estado               34100
Division             34100
Distribuidor         34100
Estructura           34100
Grado                34100
dtype: int64


In [ ]:
# Merge sales data with product and store info for better analysis
sales_data = sales_df.merge(products_df, on="Sku", how="left").merge(stores_df, on="Store_Id", how="left")

# Check for missing values
print(sales_data.isnull().sum())

# Filling missing values if necessary (e.g., fill missing prices or units with the median)
sales_data['Price'] = sales_data['Price'].fillna(sales_data['Price'].median())
sales_data['Units'] = sales_data['Units'].fillna(sales_data['Units'].median())

# Convert the 'Date' column to datetime
sales_data['Date'] = pd.to_datetime(sales_data['Date'])

# Sort data by date for time series analysis
sales_data = sales_data.sort_values('Date')


(4862029, 5)


,Sku,Date,Store_Id,Units,Price
0,4220015,2017-01-01 SO Diaria Piezas,18,6.0,15.6
1,4220015,2017-01-01 SO Diaria Piezas,24,0.0,0.0
2,4220015,2017-01-01 SO Diaria Piezas,39,0.0,0.0
3,4220015,2017-01-01 SO Diaria Piezas,44,0.0,0.0
4,4220015,2017-01-01 SO Diaria Piezas,46,0.0,0.0
